
# 🔧 MolForge Tools — Smoke Test Notebook

Este notebook prueba rápidamente que el entorno **`molforge-tools`** funciona: `numpy`, `pandas`, `pyarrow` e **`rdkit`**, además de `ipywidgets`.

**Instrucciones**:
1. Abre este `.ipynb` en JupyterLab o VS Code.
2. Selecciona el kernel **Python (molforge-tools)**.
3. Ejecuta **Run All**. Si todo va bien, verás versiones, tablas y resultados sin errores.


In [ ]:

import sys, platform
import numpy as np
import pandas as pd

# pyarrow
import pyarrow as pa
import pyarrow.parquet as pq

# rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

# ipywidgets
import ipywidgets as widgets
from IPython.display import display, Markdown

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())

print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("pyarrow:", pa.__version__)

# rdkit version is available via RDLogger or rdBase
from rdkit import rdBase
print("rdkit:", rdBase.rdkitVersion)

print("ipywidgets:", widgets.__version__)
display(Markdown("**✅ Importación correcta de todos los paquetes.**"))


In [ ]:

# NumPy: una pequeña operación determinista
a = np.arange(1, 6, dtype=float)
b = np.linspace(0, 1, 5)
c = a @ b  # producto punto
print("a =", a)
print("b =", b)
print("a·b =", c)

# pandas: DataFrame simple
df = pd.DataFrame({
    "x": np.arange(5),
    "y": np.power(np.arange(5), 2),
    "txt": list("abcde")
})
display(df)

print("✅ NumPy y pandas funcionando.")


In [ ]:

# Crear una tabla Arrow desde pandas y hacer round-trip Parquet en memoria
table = pa.Table.from_pandas(df)
import io
buf = io.BytesIO()
pq.write_table(table, buf)
buf.seek(0)
table2 = pq.read_table(buf)
df2 = table2.to_pandas()
display(df2)

assert df2.equals(df), "El DataFrame tras el round-trip debe ser igual al original"
print("✅ PyArrow in-memory Parquet round-trip OK (sin tocar disco).")


In [ ]:

# RDKit: molécula de ejemplo (Aspirina)
smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(smiles)
assert mol is not None, "RDKit no pudo parsear el SMILES"

mw = Descriptors.MolWt(mol)
rings = Chem.GetSSSR(mol)  # número de anillos
can_smi = Chem.MolToSmiles(mol, canonical=True)

# Morgan fingerprint (ECFP4 = radius 2, nBits 2048)
fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
on_bits = list(fp.GetOnBits())

print("SMILES (input):", smiles)
print("SMILES (canonical):", can_smi)
print("MolWeight:", round(mw, 4))
print("Num rings:", int(rings))
print("ECFP4 bits on:", len(on_bits), "ejemplo de índices:", on_bits[:10])

# DataFrame resumen
df_props = pd.DataFrame([{
    "smiles_in": smiles,
    "smiles_canonical": can_smi,
    "mol_weight": mw,
    "num_rings": int(rings),
    "ecfp4_bits_on": len(on_bits)
}])
display(df_props)

print("✅ RDKit básico funcionando (parseo, propiedades, fingerprint).")


In [ ]:

# Widgets: slider que muestra un cálculo y una tabla derivada
slider = widgets.IntSlider(value=3, min=0, max=10, step=1, description="n:")
out = widgets.Output()

def on_change(change):
    if change["name"] == "value":
        n = change["new"]
        with out:
            out.clear_output()
            arr = np.arange(n+1)
            dfw = pd.DataFrame({"k": arr, "k^2": arr**2})
            display(Markdown(f"**n = {n} → n^2 = {n*n}**"))
            display(dfw)

slider.observe(on_change)
display(slider)
display(out)

# Disparar una primera salida
slider.value = slider.value  # fuerza el callback

print("✅ ipywidgets renderizado (si no ves el slider, comprueba que ipywidgets está habilitado).")
